<a href="https://colab.research.google.com/github/zrsteff/CSC587/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Load the 'diabetes_test.csv' file from '/content/drive/MyDrive/diabetes_test.csv' into a pandas DataFrame, split the data into training and testing sets, and then display the shapes of the resulting training and testing sets to confirm successful loading and splitting.

## Load Data

### Subtask:
Load the 'diabetes_test.csv' file from the specified Google Drive path into a pandas DataFrame.


**Reasoning**:
To load the 'diabetes_test.csv' file from Google Drive into a pandas DataFrame, I will first import the pandas library, then specify the file path, and finally use `pd.read_csv()` to load the data.



In [6]:
import pandas as pd

file_path = '/content/drive/MyDrive/DATA/diabetes_train.csv'
df_diabetes = pd.read_csv(file_path)

print("First 5 rows of the DataFrame:")
print(df_diabetes.head())

First 5 rows of the DataFrame:
   preg  plas  pres  skin  insu  mass   pedi  age            class
0     6   148    72    35     0  33.6  0.627   50  tested_positive
1     1    85    66    29     0  26.6  0.351   31  tested_negative
2     8   183    64     0     0  23.3  0.672   32  tested_positive
3     1    89    66    23    94  28.1  0.167   21  tested_negative
4     0   137    40    35   168  43.1  2.288   33  tested_positive


In [7]:
#create test dataframe
df_diabetes_test = df_diabetes.tail(10)

print("First 5 rows of df_diabetes_test:")
print(df_diabetes_test.head())
print(f"Shape of df_diabetes_test: {df_diabetes_test.shape}")

First 5 rows of df_diabetes_test:
     preg  plas  pres  skin  insu  mass   pedi  age            class
748     3   187    70    22   200  36.4  0.408   36  tested_positive
749     6   162    62     0     0  24.3  0.178   50  tested_positive
750     4   136    70     0     0  31.2  1.182   22  tested_positive
751     1   121    78    39    74  39.0  0.261   28  tested_negative
752     3   108    62    24     0  26.0  0.223   25  tested_negative
Shape of df_diabetes_test: (10, 9)


In [8]:
df_diabetes_train = df_diabetes.iloc[:-10]

print("Shape of df_diabetes_train after removing test rows:")
print(df_diabetes_train.shape)
print("First 5 rows of df_diabetes_train:")
print(df_diabetes_train.head())

Shape of df_diabetes_train after removing test rows:
(748, 9)
First 5 rows of df_diabetes_train:
   preg  plas  pres  skin  insu  mass   pedi  age            class
0     6   148    72    35     0  33.6  0.627   50  tested_positive
1     1    85    66    29     0  26.6  0.351   31  tested_negative
2     8   183    64     0     0  23.3  0.672   32  tested_positive
3     1    89    66    23    94  28.1  0.167   21  tested_negative
4     0   137    40    35   168  43.1  2.288   33  tested_positive


**Reasoning**:
Now that `df_diabetes_train` is prepared, I will separate the features (X_train) and the target variable (y_train) and then encode the categorical target variable using `LabelEncoder` as instructed by the subtask.



In [9]:
X_train = df_diabetes_train.drop('class', axis=1)
y_train = df_diabetes_train['class']

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

print("Shape of X_train:", X_train.shape)
print("First 5 rows of X_train:")
print(X_train.head())
print("Shape of y_train:", y_train.shape)
print("First 5 values of y_train (encoded):")
print(y_train[:5])


Shape of X_train: (748, 8)
First 5 rows of X_train:
   preg  plas  pres  skin  insu  mass   pedi  age
0     6   148    72    35     0  33.6  0.627   50
1     1    85    66    29     0  26.6  0.351   31
2     8   183    64     0     0  23.3  0.672   32
3     1    89    66    23    94  28.1  0.167   21
4     0   137    40    35   168  43.1  2.288   33
Shape of y_train: (748,)
First 5 values of y_train (encoded):
[1 0 1 0 1]


## Prepare Testing Data

### Subtask:
Separate features (X_test) and the target variable (y_test) from `df_diabetes_test` for testing, and encode the target variable if it's categorical.


**Reasoning**:
I will separate the features (X_test) and the target variable (y_test) from `df_diabetes_test` and then encode the target variable using the previously fitted `label_encoder`, as per the instructions.



In [10]:
X_test = df_diabetes_test.drop('class', axis=1)
y_test = df_diabetes_test['class']

y_test = label_encoder.transform(y_test)

print("Shape of X_test:", X_test.shape)
print("First 5 rows of X_test:")
print(X_test.head())
print("Shape of y_test:", y_test.shape)
print("First 5 values of y_test (encoded):")
print(y_test[:5])

Shape of X_test: (10, 8)
First 5 rows of X_test:
     preg  plas  pres  skin  insu  mass   pedi  age
748     3   187    70    22   200  36.4  0.408   36
749     6   162    62     0     0  24.3  0.178   50
750     4   136    70     0     0  31.2  1.182   22
751     1   121    78    39    74  39.0  0.261   28
752     3   108    62    24     0  26.0  0.223   25
Shape of y_test: (10,)
First 5 values of y_test (encoded):
[1 1 1 0 0]


## Train SVM Model

### Subtask:
Initialize and train a Support Vector Machine (SVM) classifier using the prepared training data. It's often beneficial to scale features for SVMs.


**Reasoning**:
I will import necessary libraries, scale the features using `StandardScaler`, and then initialize and train a Support Vector Machine (SVM) classifier using the scaled training data, as per the subtask instructions.



In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# 1. Initialize a StandardScaler object
scaler = StandardScaler()

# 2. Fit the scaler on X_train and then transform both X_train and X_test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Shape of X_train_scaled:", X_train_scaled.shape)
print("First 5 rows of X_train_scaled (first 5 values of each feature):\n", X_train_scaled[:5, :5])
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("First 5 rows of X_test_scaled (first 5 values of each feature):\n", X_test_scaled[:5, :5])

# 3. Instantiate an SVC model
svm_model = SVC(random_state=42)

# 4. Train the SVC model
svm_model.fit(X_train_scaled, y_train)

print("SVM model trained successfully.")

Shape of X_train_scaled: (748, 8)
First 5 rows of X_train_scaled (first 5 values of each feature):
 [[ 0.64004114  0.85877541  0.15416681  0.91035357 -0.69863845]
 [-0.84307773 -1.11531885 -0.15320798  0.53327311 -0.69863845]
 [ 1.23328869  1.95549444 -0.25566624 -1.28928247 -0.69863845]
 [-0.84307773 -0.98997953 -0.15320798  0.15619264  0.11892807]
 [-1.13970151  0.51409228 -1.48516539  0.91035357  0.76254426]]
Shape of X_test_scaled: (10, 8)
First 5 rows of X_test_scaled (first 5 values of each feature):
 [[-0.24983018  2.08083376  0.05170855  0.0933459   1.04086478]
 [ 0.64004114  1.29746302 -0.3581245  -1.28928247 -0.69863845]
 [ 0.04679359  0.48275745  0.05170855 -1.28928247 -0.69863845]
 [-0.84307773  0.01273501  0.4615416   1.16174055 -0.05502225]
 [-0.24983018 -0.39461777 -0.3581245   0.21903939 -0.69863845]]
SVM model trained successfully.
